In [1]:
import pandas as pd
import time
import numpy as np
np.seterr(invalid='ignore')
import nbimporter
from PPP import PPP

prevGames = ['Game1', 'Game2', 'Game3', 'Game4', 'Game5']
Opp = 'Heat'
Team = 'Celtics'

In [2]:
players = ['brogdon', 'brown', 'grant', 'horford', 'pritchard', 'rob', 'smart', 'tatum', 'white', 'team', 'opp']

In [4]:
def load_data(players):
    
    # Establish dictionaries for new game data and overall series/season data
    new_player_data = {}
    ovr_player_data = {}
    team_opp = ['team', 'opp']
    
    for game in prevGames: # Loop over all games in series/season
        
        for player in players: # Loop over all players on roster
            
            if player == 'team': # Add all player data together to create team file for game

                try: # to load a team file for the game (helpful if already added team file, and/or rerunning for Opp file)
                    newplayerFile = (f'Team/{game + Opp}Team.csv')
                    newplayer = pd.read_csv(newplayerFile)
                    newplayer = newplayer.dropna()
                    newplayer = newplayer.set_index('Shot Type')
                    newplayer = newplayer.astype('int')
                    new_player_data[player] = newplayer # Add the existing new team file to the new_player_data dictionary
                except(FileNotFoundError):
                    # Load an empty df
                    newplayerFile = ('Team/empty.csv')
                    newplayer = pd.read_csv(newplayerFile)
                    newplayer = newplayer.set_index('Shot Type')

                    for key, df in new_player_data.items(): # Loop over the new_player_data and add together all player dataframes
                        if key not in team_opp: # Exclude the team and opponent dataframes
                            newplayer = newplayer.add(df)

                    new_player_data['team'] = newplayer # Add the created new team file to new_player_data dictionary
                    newplayer.to_csv(f'../{Opp}/Opp/{game + Team}Opp.csv') # Save team file to opponent folder for their opponent stats
                    newplayer.to_csv(f'Team/{game + Opp}Team.csv') # Save team file to team folder
                
                # For game 1, set the new team file as the overall and continue to the opponent
                if game[0:5] == 'Game1':
                    ovr_player_data['team'] = newplayer
                    newplayer.to_csv(f'!Data/Team.csv') # Write the new team file to the overall data folder
                    continue
                    
                # Load overall data for the team
                ovrplayerFile = (f'!Data/Team.csv')
                ovrplayer = pd.read_csv(ovrplayerFile)
                ovrplayer = ovrplayer.dropna()
                ovrplayer = ovrplayer.set_index('Shot Type')
                ovrplayer = ovrplayer.astype('int')
                ovr_player_data[player] = ovrplayer # Add the overall team data to the ovr_player_data dictionary
                    
                if newplayer.columns.equals(ovrplayer.columns): # Make sure the new and overall data frames can be added without error
                    ovrplayer = ovrplayer.add(newplayer)
                    ovr_player_data[player] = ovrplayer # Reassign the ovr_data_player['team'] data to the updated data
                    ovrplayer.to_csv('!Data/Team.csv') # Write the new team file to the overall data folder
                else:
                    print(f"{player.title()}'s columns do not match")
                continue # Team file is handled, so continue to the opponent file

            if player == 'opp': # Load the opponent file and add the new game data to the overall opponent file if necessary

                try: # to load the new opponent file (if running this teams data first, there will be no opponent file!!)
                    newplayerFile = (f'Opp/{game + Opp}Opp.csv')
                    newplayer = pd.read_csv(newplayerFile)
                    newplayer = newplayer.dropna()
                    newplayer = newplayer.set_index('Shot Type')
                    newplayer = newplayer.astype('int')
                    new_player_data[player] = newplayer # Add the new opponent file to the new_player_data dictionary
                except(FileNotFoundError):
                    print(f'No {player.title()} file found.')
                
                # For game 1, set the new opponent file as the overall and continue to the next player
                if game[0:5] == 'Game1':
                    ovr_player_data['opp'] = newplayer
                    newplayer.to_csv(f'!Data/Opp.csv') # Write the new opponent file into the overall data directory
                    continue
                    
                # Load overall data for the opponent
                ovrplayerFile = (f'!Data/Opp.csv')
                ovrplayer = pd.read_csv(ovrplayerFile)
                ovrplayer = ovrplayer.dropna()
                ovrplayer = ovrplayer.set_index('Shot Type')
                ovrplayer = ovrplayer.astype('int')
                ovr_player_data[player] = ovrplayer # Add the overall opponent file to the ovr_player_data dictionary

                if newplayer.columns.equals(ovrplayer.columns): # Make sure the new and overall dataframes can be added without error
                    ovrplayer = ovrplayer.add(newplayer)
                    ovr_player_data[player] = ovrplayer
                    ovrplayer.to_csv('!Data/Opp.csv') # Write the updated opponent file to the overall data directory
                else:
                    print(f"{player.title()}'s columns do not match")
                continue # Opponent file is handled, continue and exit the loop
                
            ### Loading/Adding the player files (first part of the loop)###
            try:
                newplayerFile = (f'{player.title()}/{game + Opp + player.title()}.csv') # Search the team directory for the player, then concat the game/opp/player to find csv for game
                newplayer = pd.read_csv(newplayerFile)
                newplayer = newplayer.dropna()
                newplayer = newplayer.set_index('Shot Type')
                newplayer = newplayer.astype('int')
                new_player_data[player] = newplayer # Add the new player data to the new_player_data dictionary
            except(FileNotFoundError):
                print(f'No {player.title()} file found for {game}.')
                try: # to load an overall file for the player
                    ovrplayerFile = (f'!Data/{player.title()}.csv')
                    ovrplayer = pd.read_csv(ovrplayerFile)
                    ovrplayer = ovrplayer.dropna()
                    ovrplayer = ovrplayer.set_index('Shot Type')
                    ovrplayer = ovrplayer.astype('int')
                    ovr_player_data[player] = ovrplayer # Add overall player data to the ovr_player_data dictionary
                    continue
                except(FileNotFoundError):
                    print(f'No data for {player.title()}.')
                    continue # If no player data is found, skip and go to the next player (edit for later)
            
            # For game 1, set the new player file as the overall and continue to the next player
            if game == 'Game1':
                ovr_player_data[player] = newplayer
                newplayer.to_csv(f'!Data/{player.title()}.csv')
                continue
                
            # Load overall data for the player
            try: 
                ovrplayerFile = (f'!Data/{player.title()}.csv')
                ovrplayer = pd.read_csv(ovrplayerFile)
                ovrplayer = ovrplayer.dropna()
                ovrplayer = ovrplayer.set_index('Shot Type')
                ovrplayer = ovrplayer.astype('int')
                ovr_player_data[player] = ovrplayer # Add overall player data to the ovr_player_data dictionary
            # If there is no overall data for the player, set the new player data as the overall and continue to the next player    
            except(FileNotFoundError):
                ovrplayer = newplayer
                ovr_player_data[player] = ovrplayer
                ovrplayer.to_csv(f'!Data/{player.title()}.csv')
                continue
            
            if newplayer.columns.equals(ovrplayer.columns): # Make sure the new and overall dataframes can be added without error
                ovrplayer = ovrplayer.add(newplayer)
                ovr_player_data[player] = ovrplayer
                ovrplayer.to_csv(f'!Data/{player.title()}.csv') # Write the updated overall player file to the overall data directory
            else:
                print(f"{player.title()}'s columns do not match")  
            continue
            
    return new_player_data, ovr_player_data # Return both dictionaries as a tuple

new_player_data, ovr_player_data  = load_data(players) # Unpack the tuple of dictionaries into their respective names

No Grant file found for Game1.
No Pritchard file found for Game2.
No Pritchard file found for Game3.


In [5]:
newTatumStats = PPP(new_player_data['tatum'])
newTatumStats.to_csv(f'Tatum/PPP Stats/{prevGames[-1] + Opp}.csv')
newTatumStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.49,30.9,1,70.0,1.58,23.4,75.0,4,75.0,0,N/A,1.43,38.9,66.7,3,66.7,3,66.7
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.26,32.1,2,55.6,0.87,36.2,40.0,4,50.0,1,0.0,1.8,27.8,75.0,0,N/A,4,75.0
TRANSITION,1.75,10.8,1,100.0,2.0,5.3,100.0,1,100.0,0,N/A,1.67,16.7,100.0,1,100.0,1,100.0
Attacking Closeouts,1.16,18.6,0,50.0,0.77,20.4,33.3,2,50.0,1,0.0,1.67,16.7,66.7,1,100.0,2,50.0
Catch & Shoot,1.0,8.1,0,33.3,1.0,15.8,33.3,0,N/A,3,33.3,N/A,0.0,N/A,0,N/A,0,N/A


In [6]:
tatumStats = PPP(ovr_player_data['tatum'])
tatumStats.to_csv('!PPP Stats/tatumStats.csv')
tatumStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.29,26.6,8,63.9,1.27,17.8,66.7,11,72.7,4,50.0,1.31,43.9,61.9,8,87.5,13,46.2
PNR Screener,0.67,1.7,0,33.3,0.67,2.5,33.3,1,100.0,2,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,1.25,2.2,0,50.0,3.0,0.8,100.0,0,N/A,1,100.0,0.67,4.9,33.3,1,100.0,2,0.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,2.0,0.6,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,2.0,1.6,100.0,1,100.0,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.19,27.5,6,55.0,1.19,29.0,55.6,19,63.2,8,37.5,1.2,24.6,53.8,5,60.0,8,50.0
TRANSITION,1.22,12.8,2,52.9,1.05,14.4,46.2,10,60.0,3,0.0,1.7,9.6,75.0,1,100.0,3,66.7
Attacking Closeouts,1.13,18.1,4,50.0,0.81,20.7,38.9,10,60.0,8,12.5,2.12,13.1,75.0,1,100.0,7,71.4
Catch & Shoot,1.0,5.0,0,33.3,1.0,7.6,33.3,0,N/A,9,33.3,N/A,0.0,N/A,0,N/A,0,N/A


In [7]:
newBrownStats = PPP(new_player_data['brown'])
newBrownStats.to_csv(f'Brown/PPP Stats/{prevGames[-1] + Opp}.csv')
newBrownStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.75,17.4,1,33.3,0.0,15.8,0.0,2,0.0,0,N/A,3.0,25.0,100.0,0,N/A,1,100.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.67,39.1,0,66.7,1.71,36.8,71.4,5,60.0,2,100.0,1.5,50.0,50.0,0,N/A,2,50.0
TRANSITION,1.23,21.2,0,50.0,1.33,15.8,66.7,3,66.7,0,N/A,1.06,47.0,0.0,1,0.0,0,N/A
Attacking Closeouts,0.0,4.3,0,0.0,0.0,5.3,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,1.0,13.0,0,33.3,1.0,15.8,33.3,0,N/A,3,33.3,N/A,0.0,N/A,0,N/A,0,N/A


In [8]:
brownStats = PPP(ovr_player_data['brown'])
brownStats.to_csv('!PPP Stats/brownStats.csv')
brownStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.76,11.3,2,30.8,0.22,8.9,14.3,5,20.0,2,0.0,1.45,17.2,50.0,2,0.0,4,75.0
PNR Screener,3.0,0.7,0,100.0,3.0,1.0,100.0,0,N/A,1,100.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,3.0,0.7,0,100.0,3.0,1.0,100.0,0,N/A,1,100.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,0.67,2.1,0,33.3,0.67,3.0,33.3,3,33.3,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.84,31.3,3,41.0,1.02,30.0,53.8,21,52.4,5,60.0,0.43,34.7,15.4,4,25.0,9,11.1
TRANSITION,1.07,22.5,3,51.9,1.18,16.7,60.0,13,69.2,2,0.0,0.94,37.2,41.7,5,60.0,7,28.6
Attacking Closeouts,1.29,9.9,1,61.5,0.91,10.9,50.0,9,55.6,1,0.0,2.67,7.5,100.0,1,100.0,2,100.0
Catch & Shoot,0.35,12.1,0,11.8,0.35,16.8,11.8,0,N/A,17,11.8,N/A,0.0,N/A,0,N/A,0,N/A


In [9]:
newSmartStats = PPP(new_player_data['smart'])
newSmartStats.to_csv(f'Smart/PPP Stats/{prevGames[-1] + Opp}.csv')
newSmartStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,2.0,5.9,0,100.0,2.0,7.1,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,2.0,5.9,0,100.0,2.0,7.1,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,0.61,28.7,1,33.3,0.35,20.6,0.0,2,0.0,0,N/A,1.0,66.7,100.0,1,100.0,0,N/A
Attacking Closeouts,1.03,22.8,0,33.3,1.39,20.6,50.0,2,50.0,0,N/A,0.0,33.3,0.0,1,0.0,0,N/A
Catch & Shoot,2.0,35.3,0,66.7,2.0,42.9,66.7,0,N/A,6,66.7,N/A,0.0,N/A,0,N/A,0,N/A


In [10]:
smartStats = PPP(ovr_player_data['smart'])
smartStats.to_csv('!PPP Stats/smartStats.csv')
smartStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.21,16.4,1,53.8,1.02,12.0,40.0,3,33.3,2,50.0,1.33,21.4,62.5,5,60.0,3,66.7
PNR Screener,2.5,2.2,0,100.0,2.0,2.0,100.0,1,100.0,0,N/A,3.0,2.4,100.0,0,N/A,1,100.0
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,2.5,2.2,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,2.5,4.8,100.0,1,100.0,1,100.0
DBL Ball Handler,2.0,1.1,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,2.0,2.4,100.0,1,100.0,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.51,20.4,3,75.0,1.04,13.8,50.0,4,50.0,0,N/A,1.79,28.0,87.5,4,100.0,4,75.0
TRANSITION,1.05,21.0,4,66.7,0.75,13.6,33.3,3,33.3,0,N/A,1.21,29.6,77.8,7,100.0,2,0.0
Attacking Closeouts,0.77,8.5,0,16.7,0.89,13.8,20.0,3,33.3,2,0.0,0.0,2.4,0.0,1,0.0,0,N/A
Catch & Shoot,1.42,20.9,0,47.4,1.5,36.7,50.0,0,N/A,18,50.0,0.0,2.4,0.0,0,N/A,1,0.0


In [11]:
newHorfordStats = PPP(new_player_data['horford'])
newHorfordStats.to_csv(f'Horford/PPP Stats/{prevGames[-1] + Opp}.csv')
newHorfordStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,1.5,18.2,0,50.0,N/A,0.0,N/A,0,N/A,0,N/A,1.5,50.0,50.0,0,N/A,2,50.0
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.0,18.2,0,50.0,1.0,28.6,50.0,1,100.0,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,0.67,27.3,0,33.3,0.67,42.9,33.3,1,100.0,2,0.0,N/A,0.0,N/A,0,N/A,0,N/A


In [12]:
horfordStats = PPP(ovr_player_data['horford'])
horfordStats.to_csv('!PPP Stats/horfordStats.csv')
horfordStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,0.83,11.8,1,40.0,1.0,6.1,50.0,2,50.0,0,N/A,0.75,22.2,33.3,0,N/A,3,33.3
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,0.0,2.0,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,5.6,0.0,0,N/A,1,0.0
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,2.0,2.0,0,100.0,2.0,3.0,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.0,7.8,0,50.0,1.0,12.1,50.0,3,66.7,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,1.57,15.0,0,60.0,0.0,3.0,0.0,0,N/A,1,0.0,1.81,36.9,75.0,3,100.0,1,0.0
Attacking Closeouts,1.67,5.9,0,66.7,2.0,3.0,100.0,1,100.0,0,N/A,1.5,11.1,50.0,0,N/A,2,50.0
Catch & Shoot,0.95,41.2,0,33.3,0.95,63.6,33.3,1,100.0,20,30.0,N/A,0.0,N/A,0,N/A,0,N/A


In [13]:
newBrogdonStats = PPP(new_player_data['brogdon'])
newBrogdonStats.to_csv(f'Brogdon/PPP Stats/{prevGames[-1] + Opp}.csv')
newBrogdonStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.0,50.0,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,100.0,0.0,0,N/A,2,0.0
Catch & Shoot,0.0,50.0,0,0.0,0.0,100.0,0.0,0,N/A,2,0.0,N/A,0.0,N/A,0,N/A,0,N/A


In [14]:
brogdonStats = PPP(ovr_player_data['brogdon'])
brogdonStats.to_csv('!PPP Stats/brogdonStats.csv')
brogdonStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.75,12.7,3,60.0,0.4,12.2,33.3,2,50.0,1,0.0,1.33,13.6,100.0,2,100.0,0,N/A
PNR Screener,3.0,1.6,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,3.0,4.5,100.0,0,N/A,1,100.0
DHO Ball Handler,1.5,3.2,0,50.0,0.0,2.4,0.0,0,N/A,1,0.0,3.0,4.5,100.0,0,N/A,1,100.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.15,10.9,0,0.0,0.0,9.8,0.0,3,0.0,1,0.0,0.35,13.1,0.0,0,N/A,2,0.0
TRANSITION,1.02,20.3,1,40.0,1.35,21.7,57.1,4,50.0,3,66.7,0.26,17.6,0.0,1,0.0,2,0.0
Attacking Closeouts,0.94,33.7,1,38.9,0.98,27.3,44.4,9,44.4,0,N/A,0.9,45.5,33.3,0,N/A,9,33.3
Catch & Shoot,0.0,15.9,0,0.0,0.0,22.0,0.0,0,N/A,9,0.0,0.0,4.5,0.0,0,N/A,1,0.0


In [15]:
newWhiteStats = PPP(new_player_data['white'])
newWhiteStats.to_csv(f'White/PPP Stats/{prevGames[-1] + Opp}.csv')
newWhiteStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.0,5.9,0,0.0,0.0,8.3,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,2.0,5.9,0,100.0,2.0,8.3,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,0.0,11.8,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,40.0,0.0,0,N/A,2,0.0
Attacking Closeouts,1.6,29.4,0,60.0,2.0,33.3,75.0,2,50.0,2,100.0,0.0,20.0,0.0,0,N/A,1,0.0
Catch & Shoot,1.91,43.1,1,80.0,2.22,52.7,80.0,0,N/A,5,80.0,0.0,20.0,N/A,0,N/A,0,N/A


In [16]:
whiteStats = PPP(ovr_player_data['white'])
whiteStats.to_csv('!PPP Stats/whiteStats.csv')
whiteStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.71,11.3,0,28.6,1.25,8.7,50.0,2,50.0,2,50.0,0.0,18.8,0.0,0,N/A,3,0.0
PNR Screener,3.0,1.6,0,100.0,3.0,2.2,100.0,0,N/A,1,100.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.2,8.1,0,60.0,1.33,6.5,66.7,3,66.7,0,N/A,1.0,12.5,50.0,1,100.0,1,0.0
TRANSITION,0.55,17.7,1,20.0,0.43,15.2,16.7,3,0.0,3,33.3,0.75,25.0,25.0,0,N/A,4,25.0
Attacking Closeouts,1.61,18.1,0,66.7,2.04,12.8,80.0,3,66.7,2,100.0,1.13,33.2,50.0,2,100.0,2,0.0
Catch & Shoot,1.62,40.8,2,59.1,1.69,52.9,59.1,0,N/A,22,59.1,0.0,6.2,N/A,0,N/A,0,N/A


In [17]:
newRobStats = PPP(new_player_data['rob'])
newRobStats.to_csv(f'Rob/PPP Stats/{prevGames[-1] + Opp}.csv')
newRobStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
PNR Screener,2.0,66.7,0,100.0,2.0,66.7,100.0,2,100.0,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A


In [18]:
robStats = PPP(ovr_player_data['rob'])
robStats.to_csv('!PPP Stats/robStats.csv')
robStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,1.23,40.7,4,87.5,1.55,38.8,87.5,8,87.5,0,N/A,0.0,50.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,2.0,2.8,0,100.0,2.0,3.3,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,1.6,5.2,0,100.0,1.6,6.3,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [19]:
newGrantStats = PPP(new_player_data['grant'])
newGrantStats.to_csv(f'Grant/PPP Stats/{prevGames[-1] + Opp}.csv')
newGrantStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
PNR Screener,3.0,20.0,0,100.0,3.0,20.0,100.0,0,N/A,1,100.0,N/A,N/A,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
Attacking Closeouts,1.7,35.2,0,N/A,1.7,35.2,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
Catch & Shoot,0.0,60.0,0,0.0,0.0,60.0,0.0,0,N/A,3,0.0,N/A,N/A,N/A,0,N/A,0,N/A


In [20]:
grantStats = PPP(ovr_player_data['grant'])
grantStats.to_csv('!PPP Stats/grantStats.csv')
grantStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,3.0,13.1,0,100.0,3.0,10.3,100.0,0,N/A,12,100.0,3.0,28.6,100.0,0,N/A,6,100.0
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,0.0,3.6,0,0.0,0.0,4.3,0.0,0,N/A,5,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.67,13.1,0,33.3,0.67,15.5,33.3,18,33.3,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,3.0,3.6,0,100.0,3.0,4.3,100.0,0,N/A,5,100.0,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,1.0,25.5,0,44.4,1.18,22.0,54.5,17,70.6,5,0.0,0.53,44.8,0.0,0,N/A,5,0.0
Catch & Shoot,1.29,25.5,6,58.6,1.14,25.0,47.8,0,N/A,23,47.8,2.0,28.6,100.0,6,100.0,0,N/A


In [22]:
newPritchardStats = PPP(new_player_data['pritchard'])
newPritchardStats.to_csv(f'Pritchard/PPP Stats/{prevGames[-1] + Opp}.csv')
newPritchardStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
Attacking Closeouts,0.0,100.0,1,N/A,0.0,100.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A


In [23]:
pritchardStats = PPP(ovr_player_data['pritchard'])
pritchardStats.to_csv('!PPP Stats/pritchardStats.csv')
pritchardStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.39,36.0,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,1.39,144.0,100.0,1,100.0,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.0,25.0,1,N/A,0.0,33.3,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,0.0,50.0,0,0.0,0.0,66.7,0.0,0,N/A,2,0.0,N/A,0.0,N/A,0,N/A,0,N/A


In [24]:
newTeamStats = PPP(new_player_data['team'])
newTeamStats.to_csv(f'Team/PPP Stats/{prevGames[-1] + Opp}.csv')
newTeamStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.26,14.1,2,60.0,0.95,11.2,50.0,8,50.0,0,N/A,1.62,20.0,71.4,3,66.7,4,75.0
PNR Screener,2.0,4.8,0,83.3,2.25,4.8,100.0,3,100.0,1,100.0,1.5,5.0,50.0,0,N/A,2,50.0
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.42,20.4,2,63.6,1.3,20.1,60.0,11,63.6,4,50.0,1.66,21.1,71.4,1,100.0,6,66.7
TRANSITION,1.02,12.7,2,50.0,1.02,8.2,50.0,6,50.0,0,N/A,1.01,22.2,50.0,3,66.7,3,33.3
Attacking Closeouts,1.12,16.5,0,41.2,1.33,16.1,50.0,7,42.9,3,66.7,0.71,17.5,28.6,2,50.0,5,20.0
Catch & Shoot,1.16,23.6,1,40.7,1.2,33.7,40.7,1,100.0,26,38.5,0.0,2.5,N/A,0,N/A,0,N/A


In [25]:
newOppStats = PPP(new_player_data['opp'])
newOppStats.to_csv(f'Opp/PPP Stats/{prevGames[-1] + Opp}.csv')
newOppStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.95,18.9,4,52.9,0.82,13.6,44.4,6,50.0,3,33.3,1.1,33.3,62.5,6,66.7,2,50.0
PNR Screener,2.0,6.3,0,85.7,1.83,7.4,83.3,5,80.0,1,100.0,3.0,3.3,100.0,0,N/A,1,100.0
DHO Ball Handler,1.11,8.1,1,50.0,1.17,7.4,50.0,4,50.0,2,50.0,1.0,10.0,50.0,1,0.0,1,100.0
DHO Screener,1.0,6.3,1,50.0,0.0,1.2,0.0,1,0.0,0,N/A,1.17,20.0,60.0,3,66.7,2,50.0
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.76,32.1,7,42.3,0.7,39.1,37.5,20,45.0,4,0.0,1.25,13.3,100.0,1,100.0,1,100.0
TRANSITION,1.92,9.4,0,90.0,1.75,9.2,85.7,7,85.7,0,N/A,2.33,10.0,100.0,2,100.0,1,100.0
Attacking Closeouts,1.25,3.6,1,66.7,1.0,3.7,50.0,1,0.0,1,100.0,2.0,3.3,100.0,1,100.0,0,N/A
Catch & Shoot,2.0,6.3,0,71.4,2.0,8.6,71.4,1,100.0,6,66.7,N/A,0.0,N/A,0,N/A,0,N/A


In [26]:
teamStats = PPP(ovr_player_data['team'])
teamStats.to_csv('!PPP Stats/teamStats.csv')
teamStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.02,14.2,11,48.7,0.84,9.6,41.2,24,50.0,10,20.0,1.18,23.8,54.8,19,73.7,23,39.1
PNR Screener,1.49,5.4,5,71.4,1.6,5.7,72.7,13,84.6,9,55.6,1.2,4.8,66.7,0,N/A,6,66.7
DHO Ball Handler,2.33,0.9,0,83.3,2.25,0.9,75.0,0,N/A,4,75.0,2.5,1.0,100.0,1,100.0,1,100.0
DHO Screener,1.25,0.6,0,50.0,0.0,0.2,0.0,0,N/A,1,0.0,1.67,1.4,66.7,1,100.0,2,50.0
DBL Ball Handler,1.2,0.8,0,60.0,0.67,0.7,33.3,3,33.3,0,N/A,2.0,1.0,100.0,2,100.0,0,N/A
DBL Screener,2.0,0.3,0,100.0,2.0,0.4,100.0,2,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.0,19.4,13,47.6,1.02,17.4,50.0,50,50.0,12,50.0,0.98,23.7,43.9,17,64.7,24,29.2
TRANSITION,1.1,16.8,11,50.6,1.04,14.2,48.0,37,54.1,13,30.8,1.18,22.2,54.3,17,82.4,18,27.8
Attacking Closeouts,1.05,14.2,6,44.7,0.95,14.8,42.3,38,52.6,14,14.3,1.29,13.0,50.0,5,80.0,19,42.1
Catch & Shoot,0.94,17.6,3,32.4,0.95,24.9,32.4,1,100.0,107,31.8,0.5,1.9,33.3,1,100.0,2,0.0


In [27]:
oppStats = PPP(ovr_player_data['opp'])
oppStats.to_csv('!PPP Stats/oppStats.csv')
oppStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.12,20.9,15,54.4,1.2,19.4,52.9,48,58.3,22,40.9,0.97,24.7,57.6,22,68.2,11,36.4
PNR Screener,1.13,5.9,7,64.3,1.14,6.0,60.9,19,68.4,4,25.0,1.1,5.7,80.0,1,100.0,4,75.0
DHO Ball Handler,0.82,5.3,3,30.8,0.92,5.4,33.3,5,40.0,16,31.2,0.57,5.0,20.0,2,0.0,3,33.3
DHO Screener,0.94,4.6,2,34.8,0.42,1.1,0.0,1,0.0,1,0.0,1.05,13.0,38.1,4,50.0,17,35.3
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.97,29.3,18,46.2,0.82,31.2,38.2,94,43.6,16,6.2,1.44,24.9,71.4,17,82.4,18,61.1
TRANSITION,1.56,10.9,6,78.2,1.49,9.1,75.0,23,82.6,9,55.6,1.67,15.3,82.6,13,92.3,10,70.0
Attacking Closeouts,1.31,4.9,4,66.7,1.24,5.1,61.1,15,66.7,3,33.3,1.5,4.5,83.3,4,75.0,2,100.0
Catch & Shoot,1.84,9.3,1,64.8,1.81,12.4,63.5,3,100.0,49,61.2,2.43,1.6,100.0,0,N/A,2,100.0
